This notebooks illustrates a simple method for prototyping analyses of the RCSB Protein Databank (PDB)

The data are first downloaded from the PDB archive site, these files are processed into a pandas dataframe, and then a few exploratory plots and statistics and plots are generated from the loaded dataset.

#Mount To Drive

In [1]:
#mounting
from google.colab import drive
#drive.flush_and_unmount()
drive.mount("/content/drive/")
#drive.mount("/content/drive/", force_remount=True)
!pwd

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content


Replace the directory below with the desired directory

In [2]:
#moving to current directory 
import os
%cd /content/drive/My Drive/Colab Notebooks/LSDM/
!ls

/content/drive/My Drive/Colab Notebooks/LSDM
dataset  monomers  PDB_Analysis.ipynb  PDB_API_test.ipynb  testpdb.txt


Using https://ftp.wwpdb.org/pub/pdb/data/monomers/

#Download .pdb Files from Monomer Subdirectory

May need to run more than once in case connection times out. Files which were already downloaded successfully will be skipped.

In [3]:
#Import required packages

import urllib.request
from os.path import exists
import re
from bs4 import BeautifulSoup

import requests

In [8]:
r  = requests.get("https://ftp.wwpdb.org/pub/pdb/data/monomers/")
data = r.text
soup = BeautifulSoup(data)


#for some reason soup sees duplicates of many files, just ignore

count = len(soup.find_all('a'))

flag=False

i = 0
for link in soup.find_all('a'):

    filename = link.get('href')

    #ignore anything other than the desired residue files
    if not bool(re.match(r"^[A-Z0-9]{1,3}$", filename)):

        print("Ignoring: ", filename)
        count-=1
        continue

    i+= 1


    #Don't re-download files which are already downloaded
    if exists("monomers/"+filename):

        #print("\tFile already downloaded: ", j+1, "-",  " of ", count, " : ", filename) #prints too many lines

        if flag == False:
            print("Found some files already downloaded...")
            first_pre_downloaded_file = i

        flag = True



    else:

        if flag == True:

            print("\tFiles already downloaded: ", first_pre_downloaded_file, "-", i-1, " of ", count)

        flag=False


        print("Downloading file: ", i, " of ", count, " : ", filename)

        urllib.request.urlretrieve("https://ftp.wwpdb.org/pub/pdb/data/monomers/"+filename, "monomers/"+filename)


if flag== True:
    print("\tFiles already downloaded: ", first_pre_downloaded_file, "-", i, " of ", count)




Ignoring:  ?C=N;O=D
Ignoring:  ?C=M;O=A
Ignoring:  ?C=S;O=A
Ignoring:  /pub/pdb/data/
Ignoring:  /pub/pdb/data/
Found some files already downloaded...
Ignoring:  aa-variants-v1.cif
Ignoring:  aa-variants-v1.cif
Ignoring:  aa-variants-v1.cif.gz
Ignoring:  aa-variants-v1.cif.gz
Ignoring:  components.cif
Ignoring:  components.cif
Ignoring:  components.cif.gz
Ignoring:  components.cif.gz
Ignoring:  components_iupac.cif
Ignoring:  components_iupac.cif
Ignoring:  components_iupac.cif.gz
Ignoring:  components_iupac.cif.gz
Ignoring:  het_dictionary.txt
Ignoring:  het_dictionary.txt
Ignoring:  https://www.wwpdb.org/ftp/pdb-ftp-sites
	Files already downloaded:  1 - 74024  of  74024


#Process Monomer (Residue) Files

In [9]:
#Import required packages
from string import digits
import pickle
import pandas as pd

This step takes a long time, ideally we would incorporate a way to save partial progress without having to reprocess everything if execution fails partway through.

In [ ]:
residues = []


#loop over all downloaded pdb files

num_files = len(os.listdir("monomers/"))


i=0

for filename in os.listdir("monomers/"):

    #skip any erroneous files
    if not bool(re.match(r"^[A-Z0-9]{1,3}$", filename)):

        print("\tFound erroneous file: ", filename)

        continue

    i+=1

    #print progress report every 500 files
    if (i-1)%100 == 0:
        print("Processing file: ", i, " of ", num_files, " : ", filename)

    path = os.path.join("monomers/", filename)

    monomer_file = open(path, 'r')


    atoms = []

    for index, line in enumerate(monomer_file):

        #parse the residue header
        if index==0:

            #discard the word "RESIDUE"
            id = line.split()[1]

            num_atoms = line.split()[2]


        #parse the connectivity information
        if line.startswith("CONECT"):

            #discard the word "CONECT"
            node = line.split()[1:]

            #trim digits to just keep atomic element 
            #(doesn't work because some atoms are labeled with letters instead of digits)
            #remove_digits = str.maketrans('', '', digits)
            #atom_name = node[0].translate(remove_digits)
            
            #keep only the first letter (element name)
            atom_name = node[0][0]

            num_bonds = node[1]

            bonded_atoms = []

            for bonded_atom in node[2:]:

                #keep only the first letter (element name)
                #bonded_atoms.append(bonded_atom.translate(remove_digits))
                bonded_atoms.append(bonded_atom[0])


            atom = {"atom_name":atom_name, "num_bonds":num_bonds, "bonded_atoms":bonded_atoms}

            atoms.append(atom)


    #convert atoms list of (dicts) to pandas dataframe
    atoms = pd.DataFrame(atoms)

    #print(atoms)

        
    residue = {"id":id, "num_atoms":num_atoms, "atoms":atoms}


    residues.append(residue)


    monomer_file.close()


    #if i>10: break

#convert residues list of (dicts) to pandas dataframe
residues = pd.DataFrame(residues)


#write data to file so we don't have to reprocess
with open("dataset", "wb") as f:   
    pickle.dump(residues, f) #Pickling

    print("Dataset saved to file: dataset")

#Analyze/Explore the Imported Dataset

## Load the processed data from the file

In [7]:
with open("dataset", "rb") as f:   
    dataset = pickle.load(f) # Unpickling


##Explore the loaded dataframe to quickly verify correctness

In [12]:
print(dataset)

        id num_atoms                                              atoms
0      Y7M        78     atom_name num_bonds  bonded_atoms
0        ...
1      Y7N        57     atom_name num_bonds  bonded_atoms
0        ...
2      Y7S        46     atom_name num_bonds  bonded_atoms
0        ...
3      Y7V        12     atom_name num_bonds bonded_atoms
0         ...
4      Y7Y        41     atom_name num_bonds  bonded_atoms
0        ...
...    ...       ...                                                ...
36954  0V7        27     atom_name num_bonds  bonded_atoms
0        ...
36955  0V8        27     atom_name num_bonds  bonded_atoms
0        ...
36956  0V9       119      atom_name num_bonds  bonded_atoms
0       ...
36957  0VA        49     atom_name num_bonds  bonded_atoms
0        ...
36958  0VB        57     atom_name num_bonds  bonded_atoms
0        ...

[36959 rows x 3 columns]


In [19]:
print(dataset.loc[0,:])

id                                                         Y7M
num_atoms                                                   78
atoms           atom_name num_bonds  bonded_atoms
0        ...
Name: 0, dtype: object


In [18]:
print(dataset.atoms[0])

   atom_name num_bonds  bonded_atoms
0          C         3     [N, O, O]
1          C         4  [C, C, N, H]
2          C         4  [C, C, H, H]
3          C         4  [C, C, C, H]
4          C         4  [C, H, H, H]
..       ...       ...           ...
73         H         1           [O]
74         O         1           [S]
75         O         1           [S]
76         O         2        [S, H]
77         H         1           [O]

[78 rows x 3 columns]


##Collect (and plot) statistics about the data

In [8]:
import plotly.express as px

Ensure that numeric columns are numeric and not strings

In [ ]:
dataset.total_bonds = dataset.total_bonds.astype(int)
dataset.num_atoms = dataset.num_atoms.astype(int)

Plot histogram of numer of atoms in residue

In [27]:
fig = px.histogram(dataset, x="num_atoms")
fig.show()

Plot histogram of number of bonds per atom

In [10]:
#compute average bonds per atom for each residue

total_bonds = []

for residue_index, residue in dataset.iterrows():

    print("Processing residue ", residue_index, " of ", len(dataset.index), ": ", residue.id)

    sum = 0

    for atom_index, atom in residue.atoms.iterrows():

        sum += int(atom.num_bonds)

    total_bonds.append(sum)
    
dataset['total_bonds'] = total_bonds



#write data to file so we don't have to reprocess
with open("dataset", "wb") as f:   
    pickle.dump(dataset, f) #Pickling

    print("Dataset saved to file: dataset")

Streaming output truncated to the last 5000 lines.
Processing residue  31959  of  36959 :  4ZC
Processing residue  31960  of  36959 :  4ZD
Processing residue  31961  of  36959 :  4ZE
Processing residue  31962  of  36959 :  4ZF
Processing residue  31963  of  36959 :  4ZG
Processing residue  31964  of  36959 :  4ZH
Processing residue  31965  of  36959 :  4ZJ
Processing residue  31966  of  36959 :  4ZK
Processing residue  31967  of  36959 :  4ZL
Processing residue  31968  of  36959 :  4ZM
Processing residue  31969  of  36959 :  4ZN
Processing residue  31970  of  36959 :  4ZO
Processing residue  31971  of  36959 :  4ZP
Processing residue  31972  of  36959 :  4ZQ
Processing residue  31973  of  36959 :  4ZR
Processing residue  31974  of  36959 :  4ZS
Processing residue  31975  of  36959 :  4ZT
Processing residue  31976  of  36959 :  4ZU
Processing residue  31977  of  36959 :  4ZV
Processing residue  31978  of  36959 :  4ZW
Processing residue  31979  of  36959 :  4ZX
Processing residue  31980

NameError: ignored

In [15]:
#compute bonds per atom from total bonds and number of atoms
dataset['bonds_per_atom'] = dataset.total_bonds.astype(int)/dataset.num_atoms.astype(int)

print(dataset.bonds_per_atom)

0        2.076923
1        2.105263
2        2.043478
3        2.000000
4        2.048780
           ...   
36954    2.000000
36955    2.000000
36956    1.983193
36957    2.081633
36958    2.070175
Name: bonds_per_atom, Length: 36959, dtype: float64


In [23]:
fig = px.histogram(dataset, x="total_bonds")
fig.show()

In [17]:
fig = px.histogram(dataset, x="bonds_per_atom")
fig.show()

Plot bonds per atom against total number of atoms

In [24]:
fig = px.scatter(dataset, x="num_atoms", y="bonds_per_atom")
fig.show()

Plot total number of bonds against total number of atoms

In [26]:
fig = px.scatter(dataset, x="num_atoms", y="total_bonds")
fig.show()

Plot bonds per atom against total number of bonds

In [25]:
fig = px.scatter(dataset, x="total_bonds", y="bonds_per_atom")
fig.show()